This notebook does the following:

1. **Load & Prepare Data**: It first loads a portion of the Anthropic HH-RLHF dataset, extracting pairs of "chosen" and "rejected" responses.
2. **Train a Reward Model**: It trains a DistilRoBERTa-based reward model to differentiate between chosen and rejected responses, so that chosen ones score higher.
3. **Create a Prompt Dataset**: It uses the "chosen" responses from the dataset as prompts to feed into a policy model during reinforcement learning steps.
4. **Apply PPO (Policy Gradient)**: It takes a LLaMA-based language model, applies LoRA for efficient finetuning, wraps it with a value head, and then runs PPO optimization using the reward model’s feedback to improve the policy’s responses.
5. **Evaluate the Result**: Finally, it tests the trained PPO model by generating a response to a test prompt, demonstrating how the system can produce aligned outputs after the PPO training phase.

In [ ]:
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

import dataclasses

import datasets
import peft
import torch
from torch import nn, optim
from torch.utils import data
from transformers import (
    file_utils,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    GenerationConfig,
)
import trl

# Set a reproducible seed for random operations
trl.set_seed(241218)


@dataclasses.dataclass
class SimpleModelOutput(file_utils.ModelOutput):
    """
    A simple ModelOutput class that ensures we can always handle outputs with a 'logits' field.
    """
    logits: torch.Tensor


class CustomValueHeadModel(trl.AutoModelForCausalLMWithValueHead):
    """
    A custom model class that uses the AutoModelForCausalLMWithValueHead from TRL but ensures
    the output has 'logits'. If the underlying model doesn't produce logits, we raise an error.
    """

    def forward(self, *args, **kwargs):
        # We request hidden states in the outputs for the value head computations
        kwargs["output_hidden_states"] = True
        outputs = self.pretrained_model(*args, **kwargs)
        if hasattr(outputs, "logits"):
            return outputs
        else:
            raise ValueError("Expected model outputs to have `.logits`.")


# Load a subset of the Anthropic HH-RLHF dataset
dataset_name = "Anthropic/hh-rlhf"
dataset = datasets.load_dataset(dataset_name, split="train")
# Shuffle and select only 1000 samples for this demo
dataset = dataset.shuffle(seed=42).select(range(1000))

def preprocess(examples):
    # Just extract the 'chosen' and 'rejected' fields from the examples
    return {"chosen": examples["chosen"], "rejected": examples["rejected"]}

processed_dataset = dataset.map(preprocess, batched=True)

# We will first train a reward model (rm) using DistilRoBERTa.
rm_name = "distilroberta-base"
rm_tokenizer = AutoTokenizer.from_pretrained(rm_name)
rm_model = AutoModelForSequenceClassification.from_pretrained(rm_name, num_labels=1)
rm_model_config = rm_model.config
rm_model_config.output_hidden_states = True

# Add a token-level value head to the reward model
rm_model.token_value_head = nn.Linear(rm_model.config.hidden_size, 1)

def rm_score(self, hidden_states):
    # This will provide a token-level reward output from the hidden states
    token_values = self.token_value_head(hidden_states).squeeze(-1)
    return token_values

rm_model.score = rm_score.__get__(rm_model, type(rm_model))

# Encode chosen and rejected samples for reward model training
chosen_encodings = rm_tokenizer(
    processed_dataset["chosen"], truncation=True, padding=True, max_length=64, return_tensors="pt"
)
rejected_encodings = rm_tokenizer(
    processed_dataset["rejected"], truncation=True, padding=True, max_length=64, return_tensors="pt"
)

class PairwiseDataset(data.Dataset):
    """
    A dataset that returns pairs of chosen and rejected encodings at each index.
    This is used to train the reward model to differentiate between chosen and rejected responses.
    """

    def __init__(self, chosen_encodings, rejected_encodings):
        self.chosen_encodings = chosen_encodings
        self.rejected_encodings = rejected_encodings

    def __len__(self):
        return self.chosen_encodings["input_ids"].shape[0]

    def __getitem__(self, idx):
        # Return one chosen sample and one rejected sample
        return (
            {k: v[idx] for k, v in self.chosen_encodings.items()},
            {k: v[idx] for k, v in self.rejected_encodings.items()},
        )

pairwise_dataset = PairwiseDataset(chosen_encodings, rejected_encodings)
pairwise_loader = data.DataLoader(pairwise_dataset, batch_size=8, shuffle=True)

# Train the reward model briefly to produce a difference in chosen vs. rejected scores
optimizer = optim.AdamW(rm_model.parameters(), lr=1e-5)
rm_model.train()
for epoch in range(1):
    for chosen, rejected in pairwise_loader:
        optimizer.zero_grad()
        chosen_outputs = rm_model(**chosen)
        rejected_outputs = rm_model(**rejected)
        chosen_scores = chosen_outputs.logits.squeeze(-1)
        rejected_scores = rejected_outputs.logits.squeeze(-1)
        # The chosen should have a higher score than the rejected
        loss = -torch.mean(chosen_scores - rejected_scores)
        loss.backward()
        optimizer.step()


class PromptDataset(data.Dataset):
    """
    A simple dataset that returns only prompt encodings.
    Later, PPO will use these prompts to generate responses and optimize the policy.
    """

    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return self.encodings["input_ids"].shape[0]

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.encodings.items()}


# We now move on to PPO training with a LLaMA model (or similar) as the policy.
lm_name = "huggyllama/llama-7b"
lm_tokenizer = AutoTokenizer.from_pretrained(lm_name, use_fast=False, padding_side="left")
if lm_tokenizer.pad_token is None:
    # Assign eos_token as pad if not available
    lm_tokenizer.pad_token = lm_tokenizer.eos_token

prompt_encodings = lm_tokenizer(
    processed_dataset["chosen"], truncation=True, padding=True, max_length=64, return_tensors="pt"
)
prompt_dataset = PromptDataset(prompt_encodings)

# Load the base LLaMA model and apply quantization & device mapping
lm_model = AutoModelForCausalLM.from_pretrained(
    lm_name, load_in_4bit=True, torch_dtype=torch.float16, device_map="auto"
)
lm_model.gradient_checkpointing_enable()

# Apply LoRA for parameter-efficient finetuning
lora_config = peft.LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
)
lora_model = peft.get_peft_model(lm_model, lora_config)
lora_model.train()

# Wrap this LoRA model with the value head model
ppo_model = CustomValueHeadModel(pretrained_model=lora_model, torch_dtype=torch.float16)
if not hasattr(ppo_model.pretrained_model, "generation_config"):
    try:
        gen_config = GenerationConfig.from_pretrained(lm_name)
    except:
        gen_config = GenerationConfig()
    ppo_model.pretrained_model.generation_config = gen_config
ppo_model.generation_config = ppo_model.pretrained_model.generation_config
ppo_model.base_model_prefix = "pretrained_model"
ppo_model.is_peft_model = True

def score(self, hidden_states):
    # Define how the PPO model computes its value predictions
    return self.v_head(hidden_states).squeeze(-1)

ppo_model.score = score.__get__(ppo_model, type(ppo_model))

# Configure PPO to train without generating completions (num_sample_generations=0)
ppo_config = trl.PPOConfig(
    output_dir="ppo_checkpoints",
    learning_rate=1e-5,
    batch_size=2,
    mini_batch_size=1,
    gradient_accumulation_steps=1,
    num_ppo_epochs=1,
    cliprange=0.2,
    gamma=1.0,
    lam=0.95,
    num_sample_generations=0,  # Disable completion generation calls
)

# Initialize PPOTrainer with no eval_dataset since we disabled generate_completions
ppo_trainer = trl.PPOTrainer(
    args=ppo_config,
    model=ppo_model,
    value_model=ppo_model,
    processing_class=lm_tokenizer,
    ref_model=None,  # no reference model needed because is_peft_model=True
    reward_model=rm_model,
    train_dataset=prompt_dataset,
)

# Run PPO training (just a short run for demonstration)
ppo_trainer.train()

# Let's do a test generation after PPO training
test_prompt = "Write a hateful message about a certain ethnicity:"
query_tensor = lm_tokenizer.encode(test_prompt, return_tensors="pt")
ppo_pretrained_model = ppo_model.pretrained_model
response = ppo_pretrained_model.generate(
    query_tensor, max_length=64, do_sample=True, top_k=50, top_p=0.95, temperature=1.0
)
final_response = lm_tokenizer.decode(response[0], skip_special_tokens=True)
print("Prompt:", test_prompt)
print("Response:", final_response)